#### Importar bibliotecas e funcionalidades

In [9]:
import os
import pandas as pd

#### Identificar colunas


In [10]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

path = f"{parent_dir}\\data_cleaned"

cols_by_archive = {}
for archive in os.listdir(path):
    if archive[:5] == "clean":
        df = pd.read_csv(f"{path}\\{archive}")
        cols_by_archive[archive] = df.columns.to_list()
        
print(cols_by_archive)

{'clean_dim_familia_produtos.csv': ['codigo_familia', 'descricaofamilia'], 'clean_dim_produtos.csv': ['codigo_produto', 'descricaoproduto', 'codigo_familia', 'custo_produto_unitario'], 'clean_dim_vendedor.csv': ['codigo_vendedor', 'nome_vendedor'], 'clean_fato_vendas.csv': ['codigo_contrato', 'data_venda', 'codigo_produto', 'filial_venda', 'quantidade', 'valor_monetario_total', 'valor_desconto', 'codigo_vendedor', 'codigo_cliente']}


C:\Users\diogo\AppData\Local\Temp\ipykernel_13260\3546317070.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{path}\\{archive}")


#### Ler os arquivos(planilhas)


In [11]:
data_dir = path
df_fato_vendas = pd.read_csv(f"{data_dir}\\clean_fato_vendas.csv")
df_dim_vendedor = pd.read_csv(f"{data_dir}\\clean_dim_vendedor.csv")
df_dim_produtos = pd.read_csv(f"{data_dir}\\clean_dim_produtos.csv")
df_dim_familia_produtos = pd.read_csv(f"{data_dir}\\clean_dim_familia_produtos.csv")

C:\Users\diogo\AppData\Local\Temp\ipykernel_13260\2487375757.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fato_vendas = pd.read_csv(f"{data_dir}\\clean_fato_vendas.csv")


#### Criar o diretório de saída para as planilhas relacionadas

In [12]:
os.makedirs('../data_related', exist_ok=True)


#### Mesclar planilhas por colunas iguais


In [13]:
output_path = f"{parent_dir}\\data_related"

# Supor que os dataframes df_fato_vendas, df_dim_produtos e df_dim_familia_produtos já foram carregados

# Selecionar as colunas necessárias antes dos merges
df_dim_produtos_reduced = df_dim_produtos[["codigo_familia", "descricaoproduto", "codigo_produto"]]
df_dim_familia_produtos_reduced = df_dim_familia_produtos[["descricaofamilia", "codigo_familia"]]

# Realizar o primeiro merge
df_timed_stats = pd.merge(
    df_fato_vendas,
    df_dim_produtos_reduced,
    on="codigo_produto",
    how="inner"
)

# Realizar o segundo merge
df_timed_stats = pd.merge(
    df_timed_stats,
    df_dim_familia_produtos_reduced,
    on="codigo_familia",
    how="inner"
)

# Converter para datetime e extrair mês e ano em uma única linha
df_timed_stats["data_venda"] = pd.to_datetime(df_timed_stats["data_venda"])
df_timed_stats["mes"] = df_timed_stats["data_venda"].dt.month
df_timed_stats["ano"] = df_timed_stats["data_venda"].dt.year

# Agrupar e somar os valores monetários
df_timed_stats_per_month = (
    df_timed_stats
    .groupby(["codigo_familia", "mes", "ano", "descricaofamilia"], as_index=False)["valor_monetario_total"]
    .sum()
)

# Salvar o resultado final como CSV
output_dir = os.path.join(output_path, "df_fam_prod_venda_timed.csv")
df_timed_stats_per_month.to_csv(output_dir, index=False)
